In [1]:
import pandas as pd
import numpy as np

# What we know:

#### The kaggle provided dataset contains 3 files:  
##### comment_to_score.csv
    - Contains all unranked comments. Our objective is to rank these comments based off the toxicity, with 1 being the most toxic. 

##### sample_submission.csv
    - Contains an example of how the submission should look like... Not sure if this will help with training?

##### validation_data.csv
    - According to kaggle this is the "pair rankings that can be used to validate models. Includes annotator work id and how the annotator ranked a given pair of comments"
    
# Understanding the objective
Before moving on, let's take a moment and understand what our overall objective is: 
We want to rank a list of comments base off of the level of toxicity without the use of any provided training data. However, while we do not have training data, we have provided validation data, which includes a set of toxicity rankings that might help to validate models.

# Understanding our data:
In the next few lines, we will import our data and see what we're actually dealing with and see if the data provided is actually useful. 

#### comments_to_score.csv
        - We knew how the data looked like before, but let's take a closer look and see what might need to be cleaned up  
   
   - Column Names
       - comment_id (comment identifier)
       - text (actual text)

In [2]:
# comments_to_score data
df = pd.read_csv("./data/comments_to_score.csv")
eg1 = df.text[0]
print(eg1)

"
 

Gjalexei, you asked about whether there is an ""anti-editorializing"" policy here.  There is, and it's called wikipedia:neutral point of view.  It discusses at some length  the case of what we should do when writing about a subject which most of us find repugnant.  Whilst you're not likely to get too many defenders of FGM here, the need for the policy should be clearer for articles like abortion, for instance.

If something you write is edited and you're not sure why, please continue to question such edits on the talk page.  Sometimes, you'll learn more about wikipedia policy.  Sometimes, you'll find out that some other people working on here can get it flat-out wrong ) Robert Merkel"


In [3]:
eg2 = df.text[25]
print(eg2)

"

I already gave you the source for all my edits   The Slur database, so I will not continue to play your little game. If you weren't so lazy and intent on harassment, you could use Google to search for ""Gargamel"" and ""Jew"". You get more than 400 hits including white supremacist sites and an academic paper dating to 1996. It's obviously a real slur with some usage. Nice try at being obdurate though. I'm sure there's a slur that describes that characteristic. Shyster doesn't quite cover it.  22:08, 22 Dec 2004 (UTC)"


##### took two comments from the provided dataset. We can make a few observations based off of the content of the texts:   
  
- First, punctuation. Normally, when we deal with datasets that involve texts, we would ignore punctuation. Howevever, knowing the habits of the internet, the punctuation can reflect toxicity. For instance, quotation marks have been used to represent sarcasm.
    -Building onto this, this same concept applies with capitalized letters. Generally speaking, when dealing with texts, we would try to make the letters consistent. However, given the habits of the internet, we can 
    
- I believe we can consider the second comment to be more toxic. But let's try to understand what makes it toxic: 
    - "If you weren't so lazy and intent on harassment, you could use Google to search for ""Gargamel"" and ""Jew""". 
        - This particular sentence feels targeted, with lazy being the negative description.   
        - Might want to consider pulling common words, phrases, and general patterns with the provided validation dataset as a means to start. However, keep in mind that the dataset also contains data that is not in our testing set

###### Validation_data.csv
After playing around with the data, I've come to realize the order of the comments do not matter. As described on Kaggle, this ia a collection of comments, which a person judged a pair of comments to see which one is more or less toxic (making almost binary). The comments are kinda hilarious from a third person perspective.  
  
Looking at the data on excel, I noticed that some comments are repeated, showing that it's been consistently rated as being more toxic than the other. It might be interesting to make a histogram based off of the number times a comment is rated as toxic. We can also create a list of toxic comments and have it ordered from most toxic to least toxic based off the number of counts it was rated as more toxic. 

In [4]:
vd = pd.read_csv("./data/validation_data.csv")
eg3 = vd.loc[0,:]
eg3

worker                                                      313
less_toxic              This article sucks \n\nwoo woo wooooooo
more_toxic    WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
Name: 0, dtype: object

In [5]:
eg4 = vd.loc[100,:]
eg4

worker                                                       46
less_toxic                           I'm Jim,a retarded idiot .
more_toxic     You are a nazi. \n\nYour defense of the Latin...
Name: 100, dtype: object

##### sample_submission.csv
This simply tells us how we should make our submission, it is interesting that we only care about the comment_id and the score. Although I am more interested in how the score will be developed. Is it unique or what?

In [6]:
eg5 = pd.read_csv("./data/sample_submission.csv")
eg5.loc[0:5,:]

,comment_id,score
0,114890,0.5
1,732895,0.5
2,1139051,0.5
3,1434512,0.5
4,2084821,0.5
5,2452675,0.5


### Testing out Ideas

In the previous section, I discussed ideas to explore. In this section, we will try out those ideas and see how they could apply to our model. 

In [7]:
## Find all the unique comments in the validation dataset. 

# Alright, so I want to look at this data with ease. To save time and memory, I'm going to give each unique comment a unique number
# And count the number of occurences the unique ID has. 
toxic_unique = vd.more_toxic.unique()
toxic_dict = {};
for tword in toxic_unique:
    toxic_dict[tword] = len(vd.index[vd.more_toxic == tword].tolist()) # Create a dictionary that we can use
    #I don't know how much time or memory this uses, but basicaly I am creating a dictionary which stores the number of counts per unique comment

In [8]:
toxic_df = pd.DataFrame.from_dict(toxic_dict, orient = 'index', columns = ['count'])
toxic_df = toxic_df.sort_values(by=['count'],ascending=False)
print(toxic_df[0:20])

                                                    count
 YOU SUCK \n\nI HATE YOU> YOU CAN FUCK MY ASS A...     14
STUPID SPANISH CENTRALISTSTUPID SPANISH CENTRAL...     14
MORON! The USA has only 5% of the worlds popula...     12
LAST WARNING YOU FUCKING TOSSER, IT TAKES ME 10...     12
 Imma cut off ur nutz and send em to ur gf \n\n...     12
 PEOPLE ON THIS SITE SUCK  \n\nPEOPLE ON THIS S...     12
"\n\nBastard Child\nDoes anyone know if Obama w...     12
 INTEVIEW \n\nME: ARE YOU GAY?\n\nABE GAY DASHI...     12
YOU'RE A FUCKING LIAR WHO DOESN'T KNOW HOW TO D...     12
ScJessey is a Big Fat Faggot Pinko Commie Asshole      12
 go to hell \n\ngo to hell and leave me alone          12
 Ok, what is it \n\nWhy are you such a queer.. ...     11
Why don't you just fuck off?                           11
Missrain4, I will make a rain of cum on your face.     11
 Gonna fuck you up \n\nYou little bitch. 86.178...     11
fuck you jzg \n\ni will have you removed as edi...     11
Fuck you, Smit